# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--
This project aims at developing a data model of bike sharing system for easy analysis.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
# import configparser
# from datetime import datetime
# import os
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import udf, col
# from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

The data is about bike sharing rides of two cities New York city and Bay Area. 
The data exist for August, 2019 of both the cities. 
The end use case for this is to make data available in a format suitable for analysis.
The tools I am going to use is jupyter lab.

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 
The dataset are about bike rides in New york city and Bay Area.

The data set were downloaded from 
1. https://www.citibikenyc.com/system-data
2. https://s3.amazonaws.com/baywheels-data/index.html

The information included is about bike ride and some information about rider.
For example: duration of ride, bike start station location, bike end_station_location, gender of the rider, birth_year etc.


In [2]:
# Read in the data here
bay_area = pd.read_csv("C:\\Users\\Hitesh Somani\\Documents\\udacity\\capstone_project\\baywheels_bay_area\\201908-bay-area-tripdata.csv")
nyc = pd.read_csv("C:\\Users\\Hitesh Somani\\Documents\\udacity\\capstone_project\\citibike_nyc\\201908-nyc-tripdata.csv")

In [3]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.\
# config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
# .enableHiveSupport().getOrCreate()
# df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [4]:
#write to parquet
# df_spark.write.parquet("sas_data")
# df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

- We have few missing in both the tables 
- The column names are not same

In [5]:
# Performing cleaning tasks here
bay_area.isnull().sum()/ bay_area.shape[0]

duration_sec               0.000000
start_time                 0.000000
end_time                   0.000000
start_station_id           0.000000
start_station_name         0.000000
start_station_latitude     0.000000
start_station_longitude    0.000000
end_station_id             0.000000
end_station_name           0.000000
end_station_latitude       0.000000
end_station_longitude      0.000000
bike_id                    0.000000
user_type                  0.000000
member_birth_year          0.188965
member_gender              0.188951
bike_share_for_all_trip    0.000000
dtype: float64

In [6]:
nyc.isnull().sum()/nyc.shape[0]

tripduration               0.000000
starttime                  0.000000
stoptime                   0.000000
start station id           0.000038
start station name         0.000038
start station latitude     0.000000
start station longitude    0.000000
end station id             0.000038
end station name           0.000038
end station latitude       0.000000
end station longitude      0.000000
bikeid                     0.000000
usertype                   0.000000
birth year                 0.000000
gender                     0.000000
dtype: float64

We have few missing in both the tables and also the column names are not same

In [7]:
# The bay_area table's column name are much better so we will change nyc table column name
bay_area_cols = list(bay_area.columns)
nyc_cols = list(nyc.columns)

In [8]:
for i in range(15):
    nyc = nyc.rename(columns={nyc_cols[i]: bay_area_cols[i]})

In [9]:
# bay_area table has 'bike_share_for_all_trip' column which is not available in nyc table. So dropping it.
bay_area = bay_area.drop(columns=['bike_share_for_all_trip'])

In [10]:
# Check duplication
print(bay_area.duplicated().sum())
print(nyc.duplicated().sum())

0
0


In [11]:
# Handling missing values. Since both the tables have less than 20% missing we will drop rows which have missing values
bay_area = bay_area.dropna()
nyc = nyc.dropna()

In [15]:
nyc.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender
0,393,2019-08-01 00:00:01.4680,2019-08-01 00:06:35.3780,531.0,Forsyth St & Broome St,40.718939,-73.992663,408.0,Market St & Cherry St,40.710762,-73.994004,35305,Subscriber,1996,2
1,627,2019-08-01 00:00:01.9290,2019-08-01 00:10:29.7840,274.0,Lafayette Ave & Fort Greene Pl,40.686919,-73.976682,3409.0,Bergen St & Smith St,40.686744,-73.990632,38822,Subscriber,1998,2
2,1132,2019-08-01 00:00:04.0480,2019-08-01 00:18:56.1650,2000.0,Front St & Washington St,40.702551,-73.989402,3388.0,President St & Henry St,40.682800,-73.999904,18373,Subscriber,1988,1
3,1780,2019-08-01 00:00:04.1630,2019-08-01 00:29:44.7940,479.0,9 Ave & W 45 St,40.760193,-73.991255,473.0,Rivington St & Chrystie St,40.721101,-73.991925,25002,Subscriber,1988,1
4,1517,2019-08-01 00:00:05.4580,2019-08-01 00:25:23.4550,3312.0,1 Ave & E 94 St,40.781721,-73.945940,3312.0,1 Ave & E 94 St,40.781721,-73.945940,31198,Subscriber,1965,2


In [16]:
bay_area['member_gender'].unique()

array(['Female', 'Male', 'Other'], dtype=object)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model
The conceptual data model will be star schema.
Dimension tables: Station, user and time
Fact: trips

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model
1. Make dimensional tables
2. Make fact tables

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [15]:
bay_area_cols

['duration_sec',
 'start_time',
 'end_time',
 'start_station_id',
 'start_station_name',
 'start_station_latitude',
 'start_station_longitude',
 'end_station_id',
 'end_station_name',
 'end_station_latitude',
 'end_station_longitude',
 'bike_id',
 'user_type',
 'member_birth_year',
 'member_gender',
 'bike_share_for_all_trip']

In [16]:
# Write code here
station = pd.concat([bay_area.loc[:,['start_station_id','start_station_name','start_station_latitude','start_station_longitude']].drop_duplicates(),
          bay_area.loc[:,['end_station_id','end_station_name','end_station_latitude','end_station_longitude']].drop_duplicates()], axis=0)
#station = pd.DataFrame(data = bay_area.loc[:, ['']])

c:\users\hitesh somani\documents\udacity\capstone_project\udacity_venv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
bay_area.loc[:,['start_station_id','start_station_name','start_station_latitude','start_station_longitude']].drop_duplicates()

,start_station_id,start_station_name,start_station_latitude,start_station_longitude
1,368,Myrtle St at Polk St,37.785434,-122.419622
3,104,4th St at 16th St,37.767045,-122.390833
6,323,Broadway at Kearny,37.798014,-122.405950
10,276,Julian St at The Alameda,37.332233,-121.912517
11,259,Addison St at Fourth St,37.866249,-122.299371
...,...,...,...,...
90284,425,Bird Ave at Willow St,37.311284,-121.896325
102263,397,Gish Rd at 1st St,37.361867,-121.909315
118932,396,Metro Dr at Technology Dr,37.367678,-121.918794
126082,344,16th St Depot,37.766349,-122.396292


#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.